In [101]:
import pickle
import math
import pymysql as mdb
import numpy as np
from random import shuffle
import requests

city = "SF"

master_genre_map = pickle.load( open( city+"/master_genre_map.pickle", "rb" ) )
regional_subset = pickle.load( open( city+"/regional_subset.pickle", "rb" ) )
master_genre_idf = pickle.load( open( city+"/master_genre_idf.pickle", "rb" ) )

def getArtists(evt_id):
    token = 'WGLTwENb36bAgHNc'
    query_params = { 'apikey': token,
                     'per_page': 50,
                     'page': 1}
    endpoint = 'http://api.songkick.com/api/3.0/events/'+str(evt_id)+'.json?'
    response = requests.get(endpoint, params=query_params)
    full_data = response.json()['resultsPage']['results']['event']['performance']['artist']
    print full_data

# For artist subset in one metro region
regional_venue_set = dict()
region = '26330' # LA=17835, SF=26330, Chicago=9426, SD=11086

con = mdb.connect('localhost', 'root', '', 'scenehash', autocommit=True, charset='utf8', use_unicode=True) 
cur = con.cursor()
cur.execute("SELECT evt_id, venue_name, venue_id, artist FROM events WHERE metro_id = %s" % (region))
all_events = list(cur.fetchall())

# Storage stuff
venue_list = []
venue_count = dict()
venue_ids = []
event_ids = []

artist_venue_pair = []
rand_cnt = 0
#for evtid, ven_name, ven_id, artist_name, lat, lon in all_events:
for evtid, ven_name, ven_id, artist_name in all_events:
    if ven_id is None:
        continue
    rand_cnt += 1
    if rand_cnt % 20 == 0:
        try:
            artist_venue_pair.append([ven_id, ven_name, regional_subset[artist_name]/np.linalg.norm(regional_subset[artist_name])])
            continue
        except Exception:
            continue
    if ven_name not in regional_venue_set:
        venue_list.append(ven_name)
        venue_count[ven_name] = 1
        venue_ids.append(ven_id)
        event_ids.append(evtid)
        regional_venue_set[ven_name] = np.array([0.] * len(master_genre_map))
    try:
        regional_venue_set[ven_name] +=  regional_subset[artist_name]
        venue_count[ven_name] += 1
    except Exception:
        continue

# Let's keep only those venue with 2 > events
regional_good_venues = dict()
for key, val in regional_venue_set.iteritems():
    if venue_count[key] > 3:
        regional_good_venues[key] = val

test_set = []
for artist in artist_venue_pair:
    if artist[1] in regional_good_venues:
        test_set.append([artist[0], artist[2]])

# Test set ready for prediction

from sklearn import cluster, datasets
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

def getIndex(element, my_list):
    # Create a generator
    gen = (i for i,x in enumerate(my_list) if x == element)
    for i in gen:
        return i

# Let's prepare the master_venue_vect:
master_venue_vect = []
master_venue_list = []
master_venue_locs = []

for key, val in regional_good_venues.iteritems():
    master_venue_vect.append(val)
    master_venue_list.append(key)

## Let's normalize all genre vectors for venues
for i in range(len(master_venue_vect)):
    master_venue_vect[i] = master_venue_vect[i]/np.linalg.norm(master_venue_vect[i])

X_ven = np.array(master_venue_vect)
all_of_them = []

k_rng = range(15,16)

for k in k_rng:
    k_means = cluster.KMeans(n_clusters=k,  init='k-means++', max_iter=250, n_init=20,)
    k_means.fit(X_ven) 

    # Index = cluster number, append to list of venues
    clustered_venue_names = []
    clustered_venue_ids = []
    #print k_means.labels_
    # For loop over clusters
    for cluster_num in range(k):
        temp1 = [] # names
        temp2 = [] # ids
        for i in range(len(master_venue_vect)):
            if k_means.labels_[i] == cluster_num:
                #print "Here in cluster number:", cluster_num, i, venue_list[i]
                temp1.append(master_venue_list[i])
                temp2.append(venue_ids[i])
        clustered_venue_names.append(temp1)
        clustered_venue_ids.append(temp2)
    #print clustered_venues
    
    # Let's compute some metrics
    hold_x_vals = []
    for i in range(len(clustered_venue_names)):
        hold_x_vals.append(len(clustered_venue_names[i]))
    # append average venue number, standard dev
    lng_of_them.append( np.mean(np.array(hold_x_vals)) )
    std_of_them.append( np.std(np.array(hold_x_vals)) )

# Predict
total_artists = 0
gotcha = 0
for arto in test_set:
    true_ven = arto[0]
    p_index = k_means.predict(arto[1])
    loc_venues= []
    for i in range(len(master_venue_list)):
        if k_means.labels_[i] == p_index[0]:
            ind = getIndex(master_venue_list[i], venue_list)
            loc_venues.append(venue_ids[ind])
    
    total_artists += 1
    if true_ven in loc_venues:
        gotcha += 1

        
print gotcha
print total_artists
    

'''
# Prediction

# FUnctions
def norm1(v):
    scaled_vector = []
    total_A = float(sum(v))
    if total_A == 0:
        return None
    for i in v:
        scaled_vector.append(i/total_A)
    return scaled_vector

#artists = []
#corrvens = []
#for venue_set in test_set:
#    artists.append(venue_set[3])
#    corrvens.append(venue_set[2])


token = 'WL4JBSOI4ZGXPY8FN'
query_params = { 'api_key': token,
                 'name': "",
                 'bucket': 'familiarity',
                 'bucket': 'terms'}

endpoint = 'http://developer.echonest.com/api/v4/artist/profile?'

count_success = 0
count_all = 0

for i in range(len(xartists)):

    hold_my_ven = xvenue_ids[i]
    query_params['name'] = xartists[i] #artist.lstrip().rstrip()

    response = requests.get(endpoint, params=query_params)
    full_data = response.json()

    if response.json()['response']['status']['message'] != 'Success':
        continue

    terms = full_data['response']['artist']['terms']
    #pretty_artist_names.append(full_data['response']['artist']['name'])
    genre_vec = [0] * len(master_genre_map)
    # Loop over genres, make vector of frequencies
    for term in terms:
        # Get index of genre in master_genre_map
        indx = getIndex(term['name'], master_genre_map)
        if indx is None:
            continue
        # Set corresponding vector entry to value of frequency
        genre_vec[indx] = term['frequency']
    if norm1(genre_vec) is None:
        continue
    # Set key to artist name, value is array of (normalized) freqs
    scaled_vec = np.array(norm1(genre_vec))
    # Apply tf-idf
    tfidf = scaled_vec*master_genre_idf
    # Last step, normalize vector:
    user_artist_vec = tfidf/np.linalg.norm(tfidf)
    p_index = k_means.predict(user_artist_vec)
    
    venues = []
    for i in range(len(master_venue_list)):
        if k_means.labels_[i] == p_index[0]:
            
            ind = getIndex(master_venue_list[i], venue_list)
            venues.append(venue_ids[ind])
    count_all += 1
    
    #print "This venue = %s" % hold_my_ven 
    #print "Cluster:", venues
    
    if xvenue_ids[i] in venues:
        count_success += 1

print count_success
print count_all
                    
'''


47
147


'\n# Prediction\n\n# FUnctions\ndef norm1(v):\n    scaled_vector = []\n    total_A = float(sum(v))\n    if total_A == 0:\n        return None\n    for i in v:\n        scaled_vector.append(i/total_A)\n    return scaled_vector\n\n#artists = []\n#corrvens = []\n#for venue_set in test_set:\n#    artists.append(venue_set[3])\n#    corrvens.append(venue_set[2])\n\n\ntoken = \'WL4JBSOI4ZGXPY8FN\'\nquery_params = { \'api_key\': token,\n                 \'name\': "",\n                 \'bucket\': \'familiarity\',\n                 \'bucket\': \'terms\'}\n\nendpoint = \'http://developer.echonest.com/api/v4/artist/profile?\'\n\ncount_success = 0\ncount_all = 0\n\nfor i in range(len(xartists)):\n\n    hold_my_ven = xvenue_ids[i]\n    query_params[\'name\'] = xartists[i] #artist.lstrip().rstrip()\n\n    response = requests.get(endpoint, params=query_params)\n    full_data = response.json()\n\n    if response.json()[\'response\'][\'status\'][\'message\'] != \'Success\':\n        continue\n\n    ter